# Тестирование различных провайдеров LLM

Поддерживаемые провайдеры:
- **Qwen** - локальный/удаленный Qwen API
- **YandexGPT** - Yandex Cloud LLM API
- **ChatGPT** - OpenAI API
- **Gemini** - Google Gemini API


In [7]:
import logging
import sys
import time
from pathlib import Path
from typing import Any

project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s")

print(f"📁 Корневая директория проекта: {project_root}")

📁 Корневая директория проекта: /Users/avolon/Documents/SIRIUS/T-bank_NLP_


In [8]:
from tplexity.llm_client import get_llm

print("✅ Импорты выполнены успешно")

✅ Импорты выполнены успешно


In [9]:
TEST_MESSAGES = [
    {
        "role": "system",
        "content": "Ты - полезный ассистент.",
    },
    {
        "role": "user",
        "content": "Привет!",
    },
]

GENERATION_PARAMS = {"temperature": 0.7, "max_tokens": 500}

print("Тестовый запрос:")
print(f"Системный промпт: {TEST_MESSAGES[0]['content']}")
print(f"Сообщение пользователя: {TEST_MESSAGES[1]['content']}")
print(f"Параметры генерации: {GENERATION_PARAMS}")

Тестовый запрос:
Системный промпт: Ты - полезный ассистент.
Сообщение пользователя: Привет!
Параметры генерации: {'temperature': 0.7, 'max_tokens': 500}


In [10]:
async def test_provider(
    provider: str,
    messages: list[dict],
    generation_params: dict,
) -> dict[str, Any]:
    """
    Тестирует работу LLM провайдера на тестовом запросе

    Args:
        provider (str): Название провайдера (qwen, yandexgpt, chatgpt, gemini)
        messages (list[dict]): Список сообщений в формате OpenAI (системный промпт + сообщение пользователя)
        generation_params (dict): Параметры генерации

    Returns:
        Словарь с результатами тестирования
    """
    result = {"provider": provider, "status": "unknown", "response": None, "time": 0, "error": None}

    start_time = time.time()

    try:
        print(f"Тестирование провайдера: {provider.upper()}")

        try:
            client = get_llm(provider)
            print(f"✅ Клиент инициализирован: model={client.model}")
        except Exception as e:
            error_msg = f"Ошибка инициализации клиента: {e}"
            result["error"] = error_msg
            result["status"] = "failed"
            print(f"❌ {error_msg}")
            return result

        try:
            response = await client.generate(
                messages=messages,
                temperature=generation_params.get("temperature"),
                max_tokens=generation_params.get("max_tokens"),
            )

            result["response"] = response
            result["status"] = "success"
            result["time"] = time.time() - start_time

            print(f"✅ Успешно за {result['time']:.2f}с")
            print(f"Ответ: {response}")

        except Exception as e:
            result["status"] = "failed"
            result["error"] = str(e)
            result["time"] = time.time() - start_time
            print(f"❌ Ошибка: {e}")

    except Exception as e:
        result["status"] = "failed"
        result["error"] = f"Критическая ошибка: {e}"
        result["time"] = time.time() - start_time
        print(f"❌ Критическая ошибка: {e}")

    return result

In [11]:
PROVIDERS = ["qwen", "yandexgpt", "chatgpt", "gemini"]

all_results = {}

for provider in PROVIDERS:
    result = await test_provider(
        provider=provider,
        messages=TEST_MESSAGES,
        generation_params=GENERATION_PARAMS,
    )
    all_results[provider] = result
    print()

print("Тестирование завершено")

Тестирование провайдера: QWEN
✅ Клиент инициализирован: model=QuantTrio/Qwen3-VL-30B-A3B-Instruct-AWQ


2025-11-15 19:42:57,845 - httpx - INFO - HTTP Request: POST http://195.209.210.28:8000/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-15 19:42:57,851 - tplexity.llm_client.client - INFO - ✅ [llm_client] Ответ получен от LLM
2025-11-15 19:42:57,991 - httpx - INFO - HTTP Request: POST https://llm.api.cloud.yandex.net/v1/chat/completions "HTTP/1.1 400 Bad Request"
2025-11-15 19:42:57,992 - tplexity.llm_client.client - ERROR - ❌ [llm_client] Ошибка при вызове LLM: Error code: 400 - {'error': {'message': 'Failed to parse model URI', 'type': 'invalid_request_error'}}


✅ Успешно за 0.32с
Ответ: Привет! Как я могу тебе помочь? 😊

Тестирование провайдера: YANDEXGPT
✅ Клиент инициализирован: model=gpt:///yandexgpt-lite
❌ Ошибка: Error code: 400 - {'error': {'message': 'Failed to parse model URI', 'type': 'invalid_request_error'}}

Тестирование провайдера: CHATGPT
✅ Клиент инициализирован: model=gpt-4o-mini


2025-11-15 19:42:58,238 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 401 Unauthorized"
2025-11-15 19:42:58,240 - tplexity.llm_client.client - ERROR - ❌ [llm_client] Ошибка при вызове LLM: Error code: 401 - {'error': {'message': "You didn't provide an API key. You need to provide your API key in an Authorization header using Bearer auth (i.e. Authorization: Bearer YOUR_KEY), or as the password field (with blank username) if you're accessing the API from your browser and are prompted for a username and password. You can obtain an API key from https://platform.openai.com/account/api-keys.", 'type': 'invalid_request_error', 'param': None, 'code': None}}
2025-11-15 19:42:58,352 - httpx - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/openai/chat/completions "HTTP/1.1 400 Bad Request"
2025-11-15 19:42:58,354 - tplexity.llm_client.client - ERROR - ❌ [llm_client] Ошибка при вызове LLM: Error code: 400 - [{'error': {'code':

❌ Ошибка: Error code: 401 - {'error': {'message': "You didn't provide an API key. You need to provide your API key in an Authorization header using Bearer auth (i.e. Authorization: Bearer YOUR_KEY), or as the password field (with blank username) if you're accessing the API from your browser and are prompted for a username and password. You can obtain an API key from https://platform.openai.com/account/api-keys.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

Тестирование провайдера: GEMINI
✅ Клиент инициализирован: model=gemini-2.5-flash
❌ Ошибка: Error code: 400 - [{'error': {'code': 400, 'message': 'Missing Authorization header.', 'status': 'INVALID_ARGUMENT'}}]

Тестирование завершено
